
wind_size = 60
stride_size = 5
b_size = 16
acuu- 71

<br>

wind_size = 60
stride_size = 10
b_size = 16
acuu- 70

<br>

wind_size = 120
stride_size = 10
b_size = 16
acuu- 74


wind_size = 150
stride_size = 10
b_size = 16
aucc - 72


wind_size = 180
stride_size = 10
b_size = 16
aucc - 24


wind_size = 120
stride_size = 20
b_size = 16
aucc - 30


wind_size = 120
stride_size = 5
b_size = 16
aucc - 70

# wind_size = 120, stride_size = 1, b_size = 16, learning rate=1e-4

In [ ]:
# ✅ Fully improved model with patient-wise splitting and no shuffling across patients

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from sklearn.metrics import classification_report, accuracy_score
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split

# 📦 Load dataset
path = "C:/Users/RAZER/Desktop/TEMp/New folder/KINECT_dataset_with_qor15.csv"
dataframe = pd.read_csv(path)
dataframe = dataframe[dataframe['walking_speed'] == 'Fast']
dataframe.dropna(subset=['QoR_class'], inplace=True)

# 📌 Graph structure
joints = ['PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST', 'NECK', 'CLAVICLE_LEFT', 'SHOULDER_LEFT',
          'ELBOW_LEFT', 'WRIST_LEFT', 'HAND_LEFT', 'HANDTIP_LEFT', 'THUMB_LEFT',
          'CLAVICLE_RIGHT', 'SHOULDER_RIGHT', 'ELBOW_RIGHT', 'WRIST_RIGHT', 'HAND_RIGHT',
          'HANDTIP_RIGHT', 'THUMB_RIGHT', 'HIP_LEFT', 'KNEE_LEFT', 'ANKLE_LEFT',
          'FOOT_LEFT', 'HIP_RIGHT', 'KNEE_RIGHT', 'ANKLE_RIGHT', 'FOOT_RIGHT',
          'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']

edges = [('PELVIS', 'SPINE_NAVAL'), ('SPINE_NAVAL', 'SPINE_CHEST'), ('SPINE_CHEST', 'NECK'),
         ('NECK', 'HEAD'), ('SPINE_CHEST', 'CLAVICLE_LEFT'), ('CLAVICLE_LEFT', 'SHOULDER_LEFT'),
         ('SHOULDER_LEFT', 'ELBOW_LEFT'), ('ELBOW_LEFT', 'WRIST_LEFT'), ('WRIST_LEFT', 'HAND_LEFT'),
         ('HAND_LEFT', 'HANDTIP_LEFT'), ('WRIST_LEFT', 'THUMB_LEFT'), ('SPINE_CHEST', 'CLAVICLE_RIGHT'),
         ('CLAVICLE_RIGHT', 'SHOULDER_RIGHT'), ('SHOULDER_RIGHT', 'ELBOW_RIGHT'), ('ELBOW_RIGHT', 'WRIST_RIGHT'),
         ('WRIST_RIGHT', 'HAND_RIGHT'), ('HAND_RIGHT', 'HANDTIP_RIGHT'), ('WRIST_RIGHT', 'THUMB_RIGHT'),
         ('PELVIS', 'HIP_LEFT'), ('HIP_LEFT', 'KNEE_LEFT'), ('KNEE_LEFT', 'ANKLE_LEFT'),
         ('ANKLE_LEFT', 'FOOT_LEFT'), ('PELVIS', 'HIP_RIGHT'), ('HIP_RIGHT', 'KNEE_RIGHT'),
         ('KNEE_RIGHT', 'ANKLE_RIGHT'), ('ANKLE_RIGHT', 'FOOT_RIGHT'),
         ('HEAD', 'NOSE'), ('HEAD', 'EYE_LEFT'), ('HEAD', 'EYE_RIGHT'), ('HEAD', 'EAR_LEFT'), ('HEAD', 'EAR_RIGHT')]

joint_to_idx = {j: i for i, j in enumerate(joints)}
edge_index = torch.tensor([[joint_to_idx[a], joint_to_idx[b]] for a, b in edges] + 
                          [[joint_to_idx[b], joint_to_idx[a]] for a, b in edges], dtype=torch.long).t()

# 🧱 Dataset
class WindowedSkeletonDataset(torch.utils.data.Dataset):
    def __init__(self, df, window_size=30, stride=15):
        self.samples = []
        self.labels = []
        grouped = df.groupby(['patient_id', 'trial'])
        for _, group in grouped:
            group = group.sort_values('t_uniform')
            if len(group) < window_size:
                continue
            for i in range(0, len(group) - window_size + 1, stride):
                window = group.iloc[i:i+window_size]
                x_seq, aux_seq = [], []
                for _, row in window.iterrows():
                    node_features = [[row[f'{j}_X'], row[f'{j}_Y'], row[f'{j}_Z'], row['t_uniform']] for j in joints]
                    x_seq.append(torch.tensor(node_features, dtype=torch.float))
                    aux_seq.append(torch.tensor([row['left_foot_swing'], row['right_foot_swing'], row['pelvis_pos']], dtype=torch.float))
                label = torch.tensor(window['QoR_class'].mode().values[0], dtype=torch.float)
                self.samples.append((torch.stack(x_seq), torch.stack(aux_seq), label))
                self.labels.append(int(label.item()))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

# 🔀 Patient-wise split
unique_ids = dataframe['patient_id'].unique()
train_ids, test_ids = train_test_split(unique_ids, test_size=0.3, random_state=42)

train_df = dataframe[dataframe['patient_id'].isin(train_ids)]
test_df  = dataframe[dataframe['patient_id'].isin(test_ids)]







#---------------------------------------------------------------------------------------------------------------------------------------------------------

wind_size = 120
stride_size = 10
b_size = 16

#---------------------------------------------------------------------------------------------------------------------------------------------------------






train_dataset = WindowedSkeletonDataset(train_df,window_size=wind_size, stride=stride_size)
test_dataset  = WindowedSkeletonDataset(test_df,window_size=wind_size, stride=stride_size)

# 📦 Dataloaders (no shuffle)
train_loader = DataLoader(train_dataset, batch_size=b_size)
test_loader  = DataLoader(test_dataset, batch_size=b_size)

# 🧠 Model
class WindowedSTGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels):
        super().__init__()
        self.gcn = GCNConv(in_channels, hidden_channels)
        self.lstm = nn.LSTM(hidden_channels, hidden_channels, batch_first=True)
        self.aux_fc = nn.Linear(3, hidden_channels)
        self.classifier = nn.Linear(2 * hidden_channels, 1)

    def forward(self, x_seq, aux_seq):
        B, T, N, feat_dim = x_seq.shape
        x_seq = x_seq.view(B * T * N, feat_dim)
        full_edge_index = torch.cat([edge_index + b * N for b in range(B * T)], dim=1).to(x_seq.device)
        x = F.relu(self.gcn(x_seq, full_edge_index))
        x = x.view(B, T, N, -1).mean(2)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        aux = F.relu(self.aux_fc(aux_seq.mean(1)))
        return self.classifier(torch.cat([x, aux], dim=-1)).squeeze()

# ✅ Training
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = WindowedSTGCN(in_channels=4, hidden_channels=128).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCEWithLogitsLoss()

for epoch in range(1, 100):
    model.train()
    total_loss, correct, total = 0, 0, 0
    for x_seq, aux_seq, y in train_loader:
        x_seq, aux_seq, y = x_seq.to(device), aux_seq.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x_seq, aux_seq)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * y.size(0)
        correct += ((torch.sigmoid(out) >= 0.5).float() == y).sum().item()
        total += y.size(0)
    print(f"Epoch {epoch:03d} | Loss: {total_loss / total:.4f} | Accuracy: {correct / total:.4f}")

# 🧪 Final Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for x_seq, aux_seq, y in test_loader:
        x_seq, aux_seq = x_seq.to(device), aux_seq.to(device)
        out = torch.sigmoid(model(x_seq, aux_seq))
        pred = (out >= 0.5).float()
        y_true.extend(y.cpu().numpy())
        y_pred.extend(pred.cpu().view(-1).numpy())


print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=["Good (0)", "Not Good (1)"]))

Epoch 001 | Loss: 21.9099 | Accuracy: 0.4794
Epoch 002 | Loss: 6.0230 | Accuracy: 0.4979
Epoch 003 | Loss: 5.6886 | Accuracy: 0.5095
Epoch 004 | Loss: 5.7203 | Accuracy: 0.5129
Epoch 005 | Loss: 5.7078 | Accuracy: 0.5124
Epoch 006 | Loss: 5.6835 | Accuracy: 0.5106
Epoch 007 | Loss: 5.6514 | Accuracy: 0.5098
Epoch 008 | Loss: 5.6203 | Accuracy: 0.5111
Epoch 009 | Loss: 5.6079 | Accuracy: 0.5165
Epoch 010 | Loss: 5.5964 | Accuracy: 0.5144
Epoch 011 | Loss: 5.5660 | Accuracy: 0.5101
Epoch 012 | Loss: 5.5449 | Accuracy: 0.5039
Epoch 013 | Loss: 5.5786 | Accuracy: 0.5023
Epoch 014 | Loss: 5.6621 | Accuracy: 0.4990
Epoch 015 | Loss: 5.5768 | Accuracy: 0.5041
Epoch 016 | Loss: 5.5684 | Accuracy: 0.5064
Epoch 017 | Loss: 5.5527 | Accuracy: 0.5090
Epoch 018 | Loss: 5.5455 | Accuracy: 0.5085
Epoch 019 | Loss: 5.5374 | Accuracy: 0.5080
Epoch 020 | Loss: 5.5421 | Accuracy: 0.5080
Epoch 021 | Loss: 5.5403 | Accuracy: 0.5077
Epoch 022 | Loss: 5.5397 | Accuracy: 0.5077
Epoch 023 | Loss: 5.5363 | Accu

c:\Users\RAZER\anaconda3\envs\torch_on\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\RAZER\anaconda3\envs\torch_on\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\RAZER\anaconda3\envs\torch_on\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result)

In [8]:
from collections import defaultdict
import pandas as pd
import torch

model.eval()
patient_results = defaultdict(list)

# Align test_df index to help lookup patient IDs
test_df = test_df.reset_index(drop=True)

with torch.no_grad():
    for i in range(len(test_dataset)):
        x_seq, aux_seq, y = test_dataset[i]
        patient_id = test_df.iloc[i]['patient_id']
        x_seq, aux_seq = x_seq.unsqueeze(0).to(device), aux_seq.unsqueeze(0).to(device)
        output = model(x_seq, aux_seq)
        prob = torch.sigmoid(output).item()
        pred_label = int(prob >= 0.5)
        true_label = int(y.item())

        patient_results[patient_id].append({
            'pred': pred_label,
            'true': true_label,
            'prob': prob
        })

# Build summary per patient
summary = []
for pid, records in patient_results.items():
    preds = [r['pred'] for r in records]
    trues = [r['true'] for r in records]
    probs = [r['prob'] for r in records]

    correct_count = sum(p == t for p, t in zip(preds, trues))
    total = len(records)
    valid_percentage = 100 * correct_count / total
    invalid_percentage = 100 - valid_percentage

    summary.append({
        'PatientID': pid,
        'Predicted_QoR_class': max(set(preds), key=preds.count),
        'Actual_QoR_class': max(set(trues), key=trues.count),
        'valid_percentage': round(valid_percentage, 2),
        'invalid_percentage': round(invalid_percentage, 2),
        'average_model_output': round(sum(probs) / len(probs), 6)
    })

# Convert to DataFrame and view
report_df = pd.DataFrame(summary).sort_values(by='PatientID').reset_index(drop=True)
display(report_df)

# Save if needed
# report_df.to_csv("QoR_Per_Patient_Report.csv", index=False)


,PatientID,Predicted_QoR_class,Actual_QoR_class,valid_percentage,invalid_percentage,average_model_output
0,001-LO,0,1,39.42,60.58,0.390325
1,005-GP,0,0,58.72,41.28,0.329031
2,006-TR,0,0,89.06,10.94,0.115922
3,010-RA,0,1,53.30,46.70,0.043899
4,011-CB,0,0,59.44,40.56,0.159074
5,013-MS,0,1,41.16,58.84,0.211195
6,018-OS,0,1,6.02,93.98,0.070774
7,020-MM,0,0,65.44,34.56,0.069975
8,025-PS,0,0,100.00,0.00,0.000419
9,033-JF,0,0,85.89,14.11,0.039994
